# Markov chain Monte Carlo: Amortized Approximate Likelihood Ratios

In [1]:
import hypothesis
import torch
import numpy as np
import matplotlib.pyplot as plt

## Forward model

In [2]:
from hypothesis.simulation import Simulator

class NormalSimulator(Simulator):
    
    def __init__(self):
        super(NormalSimulator, self).__init__()
        
    def forward(self, inputs):
        return torch.randn(inputs.size(0), 1) + inputs

simulator = NormalSimulator()

## Prior

In [3]:
from torch.distributions.uniform import Uniform

prior = Uniform(-30, 30)

## Ratio estimator architecture

In [4]:
from hypothesis.nn import BaseConditionalRatioEstimator

class RatioEstimator(BaseConditionalRatioEstimator):
    
    def __init__(self):
        super(RatioEstimator, self).__init__()
        self.network = torch.nn.Sequential(
            torch.nn.Linear(2, 64),
            torch.nn.ELU(),
            torch.nn.Linear(64, 64),
            torch.nn.ELU(),
            torch.nn.Linear(64, 64),
            torch.nn.ELU(),
            torch.nn.Linear(64, 1))
        
    def forward(self, inputs, outputs):
        log_ratio = self.log_ratio(inputs, outputs)
        
        return log_ratio.sigmoid(), log_ratio
        
    def log_ratio(self, inputs, outputs):
        inputs = inputs.view(-1, 1)
        outputs = outputs.view(-1, 1)
        x = torch.cat([inputs, outputs])
    
        return self.network(x)

## Training the ratio estimator

In [5]:
from hypothesis.util.data import SimulatorDataset

ratio_estimator = RatioEstimator()
dataset = SimulatorDataset(simulator, prior)

## Validating the ratio estimator

## Posterior inference using MCMC